#### Библиотеки

In [ ]:
import sys

sys.path.append("..")

import warnings

import pandas as pd
import torch
from datasets import load_dataset
from transformers import (
    DistilBertForSequenceClassification,
    DistilBertTokenizerFast,
)

from datasets import load_from_disk
import os

from warp.utils.misc import seed_everything
from warp.utils.data import df_self_product, prepare_reward_dataset
from warp.constants import DATASET_DIR, CONFIG_DIR, MODEL_DIR
from pathlib import Path

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2
%cd ..

device = "cuda" if torch.cuda.is_available() else "cpu"
device

Для работы нам понадобится всего одна модель, а именно берт. Поскольку у меня он есть локально, путей я указал тоже 2, при желании можно откатиться, линию я закомментил

In [5]:
model_name = Path(MODEL_DIR, "distilbert/distilbert-base-cased")
# model_name = "distilbert/distilbert-base-cased"

reward_tokenizer = DistilBertTokenizerFast.from_pretrained(
    model_name, max_length=512
)
reward_model = DistilBertForSequenceClassification.from_pretrained(
    model_name
).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /home/ivanov.dko/projects/test/rl/artifacts/models/distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Датасет, как и просили, достанем из "imdb". Нам нужен только трейн, но это всё равно игрушечный ноутбук, так что не важно

In [4]:
train, test = load_dataset("imdb", split=["train", "test"])
train, test = [pd.DataFrame(dataset) for dataset in [train, test]]
train

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


Далее по условию нужно достать множество всех пар положительных и отрицательных отзывов. Для этого у меня есть отдельная функция. Должна работать быстро. Не должна крашиться по памяти, хотя там есть, что править. В частности все 125кк семплов нам не нужны, я взял лишь подмножество, и как это сделать эффективно - пока что вопрос. Подробнее в `warp.utils.data.df_self_product`

In [5]:
sets = (
    df_self_product(train, partition_col="label")
    .sample(1000)
    .rename({"text_0": "chosen", "text_1": "rejected"})
)

In [15]:
sets.sample(1).to_dict(as_series=False)

{'chosen': ['I have two very major complaints regarding this film.<br /><br />1. That my local rental store shelved what is very clearly a soft core porn in the "suspense" category. (Had I known what it was, I would not have wasted my time renting it in the first place. And yes, this movie is a soft core porn.)<br /><br />2. The title has nothing to do with the movie. No one in this movie does anything that is either deviant or obsessive, let alone a combination of the two.<br /><br />Actually, make that three major complaints:<br /><br />3. That I for some reason watched the movie long enough to discover point number two on this list. Boy do I regret that. Stay away from this movie. Learn from my mistake. This movie is valueless on virtually every level.'],
 'rejected': ["I love this movie. My friend Marcus and I were browsing the local Hastings because we had an urge to rent something we had never seen before and stumbled across this fine film. We had no idea what it was going to be 

Ну, как мы видим метки немножко перепутались, но так даже прикольнее, мы же не обсуждали, какой именно alignment мы хотим. Правится это элементарно - у меня выше есть `rename`

Дальше нам нужно подготовить датасет. Там всё просто - токенизируем каждый пример и пишем в датасет. Примеры функции уже есть в `huggingface`, но я её немножко модернизировал. Параллелить смысла нет, быстрый токенайзер уже действительно быстрый

In [20]:
reward_dataset = prepare_reward_dataset(
    sets.to_dict(as_series=False), tokenizer=reward_tokenizer
)
reward_dataset = reward_dataset.train_test_split(test_size=0.2)
reward_dataset

2024-08-05 13:23:25.184 | INFO     | warp.utils.data:prepare_reward_dataset:122 - Starting tokenizing `chosen`
2024-08-05 13:23:25.623 | INFO     | warp.utils.data:prepare_reward_dataset:122 - Starting tokenizing `rejected`


DatasetDict({
    train: Dataset({
        features: ['input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
        num_rows: 800
    })
    test: Dataset({
        features: ['input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
        num_rows: 200
    })
})

In [21]:
reward_dataset["train"][0]

{'input_ids_chosen': tensor([  101,   146,  1138,  1309,  2542,   170,  2523,  1107,  1177,  1376,
          1159,   119,  1109,  1178, 19718,  1108,  1103,  2698,  1977,  3053,
          1113,  1103,  4173,  2587,   119,  1135,  1108,  1176,  2903,   170,
          9874,  1666, 13979,  1273,   119,  1247,  1108,  5544,  1185,  1948,
          1111,  7570,   117,  7973,   117,  3741,   117,  2450,   117,  5444,
         13814,  1116,   117, 11884,   117,  5681,   119,   119,   119,  2048,
           117,  1175,  1108,  7284,  1185,  1642,  1719,   106,   146,  1444,
          1106,  3593,  1995,  2442,  1104,  7368,   119,   119,   119,  6304,
           117,  9684,   117,  9210,   117,   192,  8127,  6428,   117, 25096,
          1158,   117,  1106,  3740,  8163,   117, 16516,  9436,  1361,   117,
          9468,  5613,  2285,   117,  4809,  4777,   117,  1121,  2553,   117,
         22852,   117, 22593, 25936,  1183,   117,   178, 23143,   117,   194,
          8474,  1183,   117, 23

Ну а дальше остаётся только прожать конфиг. Они сохранены у меня в отдельных файлах, так что нужно их оттуда достать. Я по итогу отказался от использования адаптера, потому что модель не обучена вся, но опция такая есть

In [23]:
from peft import LoraConfig
from trl import RewardTrainer, RewardConfig
from omegaconf import OmegaConf

import os

peft_params = OmegaConf.to_container(
    OmegaConf.load(Path(CONFIG_DIR, "reward/peft.yaml"))
)["peft"]
peft_config = LoraConfig(**peft_params)

reward_trainer_params = OmegaConf.to_container(
    OmegaConf.load(Path(CONFIG_DIR, "reward/trainer.yaml"))
)["trainer"]
reward_config = RewardConfig(**reward_trainer_params)

In [24]:
reward_trainer_params

{'output_dir': 'outputs/',
 'per_device_train_batch_size': 16,
 'num_train_epochs': 2,
 'gradient_checkpointing': True,
 'gradient_checkpointing_kwargs': {'use_reentrant': False},
 'learning_rate': 2e-05,
 'report_to': 'none',
 'remove_unused_columns': False,
 'optim': 'adamw_torch',
 'logging_steps': 500,
 'max_length': 512,
 'load_best_model_at_end': True,
 'evaluation_strategy': 'steps'}

Запускать я конечно ничего не буду, если очень хочется, то можно сделать вот так, но предварительно стоит пройтись глазами по конфигам, если что-то не устраивает - поменять. Сейчас там стоит то, с чем обучался я. Предварительно конечно стоит поставить `poetry`, но можно и убрать `!poetry run`, если есть все зависимости, то оно запустится

In [ ]:
!poetry run python warp/train_reward.py --config-name reward_config

In [ ]:
trainer = RewardTrainer(
    model=reward_model,
    args=reward_config,
    tokenizer=reward_tokenizer,
    train_dataset=reward_dataset["train"],
    eval_dataset=reward_dataset["test"],
    peft_config=peft_config,
)

trainer.train()